In [4]:
import re
from newspaper import Article, ArticleException

def crawl_newspaper(url):
    """기사 링크를 입력으로 받아 제목, 내용 그리고 링크를 파싱하여 dictionary 형태로 반환하는 함수"""
    
    try:
        article = Article(url,
                          language='ko',
                          memorize_articles=False,
                          fetch_images=False,
                          skip_bad_cleaner=True)
        article.download()
        article.parse()

        article_data = {}
        article_data['title'] = article.title.strip()  # 기사 제목
        article_data['text'] = article.text.replace("\n", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
        article_data['url'] = url  # 기사 링크

        return article_data
    except ArticleException as AE:
        raise AE
    
    
tmp_article = crawl_newspaper("https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003376441")

for k, v in tmp_article.items():
    print(k, ":", v[:100])
    

    import requests
from bs4 import BeautifulSoup

def get_newspaper_list(query,  # 검색어
                       date=None,  # 기사 일자
                       start_page=1  # 뉴스 리스트 페이지
                      ):
    """네이버에서 검색어를 이용해 해당 기간 내의 뉴스 리스트를 반환하는 함수"""
    
    url = 'https://search.naver.com/search.naver?&where=news&query={query}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&start={start_page}'  # query와 기간을 조건으로 한 검색 url 
    if not date:
        date = datetime.now()  # 오늘 날짜 instance 생성
        date = date.strftime("%Y.%m.%d")  # 날자 instance를 문자열 형태로 변환 ex) 2019.06.30
    
    res = requests.get(url.format(query=query, 
                                  start_date=date, 
                                  end_date=date, 
                                  start_page=(start_page - 1) * 10 + 1))
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.text, 'lxml')
    
    # 최대 기사 페이지 수를 확인
    pages = [tag.get_text() for tag in soup.select(".paging > a, .paging > strong")] # 이전페이지,1,2,3페이지
    max_page = 0
    for page in pages:
        try:
            page = int(page)
            max_page = max(page, max_page)
        except:
            continue
            
    if start_page > max_page:  # 더이상 페이지가 없을 경우 빈 리스트를 반환
        return []
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출
    link_list = [a['href'] for a in soup.select(".news > ul.type01 > li > dl > dt > a")]  
    return link_list


from datetime import datetime, timedelta
from time import time

def crawl_newspaper_by_query(query,  # 검색어 키워드
                             start_date=None,
                             end_date=None,
                             max_page=10):
    
    
    article_list = []  # 기사 리스트
    crawled_url = set()  # 검색한 기사 중복 검색을 위한 변수
        
    if not start_date:
        start_date = datetime.now()  # 오늘 날짜 instance 생성
    else:
        start_date = datetime.strptime(start_date, "%Y.%m.%d")  # 2019.06.30과 같은 문자열을 python instance로 변경
        
    if not end_date:
        end_date = datetime.now()
    else:
        end_date = datetime.strptime(end_date, "%Y.%m.%d")
    
    while start_date < end_date:
        print("Date: ", end_date)
        for cur_page in range(1, max_page):
            print("Page: {}".format(cur_page))
            
            # end_date 날짜의 기사 링크 수집
            link_list = get_newspaper_list(query, end_date.strftime("%Y.%m.%d"), start_page=cur_page)

            if not link_list:  # 더이상 수집할 뉴스가 없을 경우
                break
            
            for link in link_list:
                # 중복 수집 방지
                if link in crawled_url:
                    continue
                else:
                    crawled_url.update([link])
                
                try:
                    article = crawl_newspaper(link)
                    article['date'] = start_date
                    article_list.append(article)
                except ArticleException as AE:
                    continue
        
        end_date = end_date - timedelta(days=1)  # 수집할 날짜를 하루 전으로 변경
    return article_list



title : 북한땅 10여m 올라간 트럼프… “군사분계선 넘은 것 영광” [남북미 정상, 사상 첫 판문점 회동]
text : 김정은 북한 국무위원장과 도널드 트럼프 미국 대통령이 30일 오후 3시47분 판문점 공동경비구역(JSA) 군사분계선 북측 지역으로 넘어간 뒤 다시 남측으로 내려오고 있다.판문점=연
url : https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=022&aid=0003376441


In [5]:
start = time()
articles = crawl_newspaper_by_query("sk하이닉스 주가하락", "2019.06.01", "2019.07.15", max_page=10)
print(time() - start)
print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[0])




Date:  2019-07-15 00:00:00
Page: 1
Page: 2
Date:  2019-07-14 00:00:00
Page: 1
Page: 2
Date:  2019-07-13 00:00:00
Page: 1
Page: 2
Date:  2019-07-12 00:00:00
Page: 1
Page: 2
Date:  2019-07-11 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-10 00:00:00
Page: 1
Page: 2
Date:  2019-07-09 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-08 00:00:00
Page: 1
Page: 2
Page: 3
Page: 4
Date:  2019-07-07 00:00:00
Page: 1
Page: 2
Date:  2019-07-06 00:00:00
Page: 1
Page: 2
Date:  2019-07-05 00:00:00
Page: 1
Page: 2
Date:  2019-07-04 00:00:00
Page: 1
Page: 2
Date:  2019-07-03 00:00:00
Page: 1
Page: 2
Date:  2019-07-02 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-07-01 00:00:00
Page: 1
Page: 2
Date:  2019-06-30 00:00:00
Page: 1
Date:  2019-06-29 00:00:00
Page: 1
Page: 2
Date:  2019-06-28 00:00:00
Page: 1
Page: 2
Date:  2019-06-27 00:00:00
Page: 1
Page: 2
Date:  2019-06-26 00:00:00
Page: 1
Page: 2
Page: 3
Date:  2019-06-25 00:00:00
Page: 1
Page: 2
Date:  2019-06-24 00:00:00
Page: 1
Page: 2
Date: 

C:\Users\Administrator\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


NameError: name 'sentence' is not defined

In [7]:
from konlpy.tag import Okt

# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)

sentence = "SK텔레콤-LG전자, 5G로 클라우드와 로봇 결합했다… 로봇산업 혁신 기대"

words = tagger.pos(sentence, 
                   norm=True,  # 정규화 - 입니다 ㄹㄹ -> 입니다로 모아주는
                   stem=True)  # 어근화 - 원형으로 바꾸는 작업
for w, t in words:
    print(w, t)
    
    
from collections import Counter

words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set(["글자"])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..

for i, article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        
    words = tagger.pos(article['text'], norm=True, stem=True)
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    
    
print(words_list[0])

vocab = sorted([w for w, freq in vocab.most_common(10000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    
    # 기사 내 서로 다른 단어 개수가 10개 이상인 기사만 사용
    if len(words) > 10:
        new_words_list.append((words, article))
        

        from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix.toarray()[:10, :20])

SK Alpha
텔레콤 Noun
- Punctuation
LG Alpha
전자 Noun
, Punctuation
5 Number
G Alpha
로 Noun
클라우드 Noun
와 Josa
로봇 Noun
결합 Noun
하다 Verb
… Punctuation
로봇산업 Noun
혁신 Noun
기대 Noun
0
100
200
(['News', 'DB', '코스피', '보합', '마감', '코스닥', '외국인', '기관', '동반', '매도', '가까이', '하락', '코스피', '코스닥', '거래', '일만', '하락', '세로', '전환', '코스피', '거래', '비교', '포인트', '마감', '코스피', '지수', '오전', '거래', '일과', '비교', '포인트', '기도', '이내', '반등', '낙폭', '일본', '한국', '소재', '수출', '규제', '여파', '가격', '상승세', '타고', '소식', '전자', 'SK', '하이닉스', '초반', '하락', '상승세', '거래', '영향', '외국인', '기관', '각각', '매수', '개인', '홀로', '매도', '외국인', '기관', '수로', '개인', '도로', '각각', '전환', '코스피', '시가총액', '상위', '종목', 'SK', '하이닉스', '지주', 'LG', '화학', 'POSCO', '삼성', '전자', '전자', 'SK', '텔레콤', '상승', '셀트리온', '현대차', '하락', '업종', '별로', '은행', '전기', '전자', '기가스', '화학', '통신업', '상승', '나머지', '업종', '모두', '하락', '하락', '의약품', '섬유', '의복', '건설업', '유통업', '음식', '료업', '코스닥', '거래', '대비', '포인트', '마감', '코스닥', '거래', '거래', '다시', '바이오', '주가', '동반', '하락', '결과', '지난', '이어진', '거래', '연속', '상승', '날로', '외국인', '기관', '각각'

In [8]:
from sklearn.decomposition import NMF

K = 10  # topic 개수
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (298, 10)
H shape: (10, 3931)


In [9]:
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
        
    print("\n", "="*20)

0th topic
거래 중이 거래량 백만원 기록 전일 주가 상승 금은 대비 고가 저가 가격 대금 주식 하락 수준 각각 등락 종목 
1th topic
기사 전자 삼성 금융 투자자 LG SK 외국인 실시간 알고리즘 중점 경로 이중 동생 공동 AI 하이닉스 작성 종목 전문 
2th topic
일본 규제 수출 소재 반도체 업체 생산 국내 디스플레이 한국 정부 주가 기업 차질 조치 이번 지스트 영향 국산 제품 
3th topic
지수 포인트 하락 마감 코스피 기관 매도 외국인 거래 오른 셀트리온 전장 개인 반면 출발 코스닥 매수 전날 미국 종목 
4th topic
반도체 순이익 장비 테라 시가총액 분석 기록 종목 매출 영업 이익 마이크로 거래 식수 주중 PER 리포트 분류 컨텍 순위 
5th topic
반도체 화웨이 하반기 실적 전망 연구원 업황 분쟁 전자 미국 주가 무역 수요 메모리 가격 디램 회복 마이크론 반등 재고 
6th topic
종목 개인 매수 수익률 평균 상위 투자자 외국인 쇼핑 주가 이마트 지온 기간 아난 한국전력 반면 제외 대부분 상반기 기아차 
7th topic
대차 잔고 주식 공매도 차입 기준 급증 투자자 지난해 무역 지난달 선행 코스피 지불 보물 세이브 수수료 결제 상환 투자신탁 
8th topic
KODEX 레버리지 버스 선물 기관 기사 코스닥 투자자 금융 지주 전기 종목 실시간 알고리즘 경로 중점 공동 동생 이중 AI 
9th topic
영업 분기 이익 실적 낸드 대치 감소 SK 투자 예상 치가 발생 목표 NAND 증권 라며 하이닉스 전망 확대 수요 


In [10]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1]['title'])
    print("\n", "="*20)

0th topic
저점 매수 나선 기관투자가, 증시 구원투수로
07일, 기관 거래소에서 삼성전기(-0.93%), 롯데케미칼(-3.94%) 등 순매도
05일, 거래소 기관 순매수상위에 운수장비 업종 2종목

1th topic
"SK하이닉스 주가 상승 가능", 무역분쟁 완화하면 업황 빠르게 회복
'국내 50대 부자' 재산 16.7% 감소…감소폭 가장 큰 부자는?
"기다리면 오를까?" 반도체 업황 개선에 건 기대

2th topic
코스피, 미중 무역분쟁 재협상 불확실성에 미국 유럽관세 부과 영향 하락-2120선 유지
韓日 무역분쟁 확산 우려감에 증시 '검은 월요일'
日 반도체 규제에 관련주 ‘출렁’... "단기 생산 차질" 우려

3th topic
[코스피 거래량 Top 10 :장중시황] SG세계물산 0.73% 상승, SH에너지화학, KODEX 200선물인버스2X, SG충방, 이아이디 주가 순으로 14일 거래량 상위 기록.
윤종규·조용병 해외 IR 나서자 템플턴·블랙록 투자로 호재
지나치게 낮아지기 시작한 시장 기대치·비중 확대 고려할 시기

4th topic
마이크로컨텍솔, 3585원까지 올랐으나 3520원 후퇴
테라셈, 장초반 1905원 상승 이후 후퇴…현재가 1775원
D램가격 10개월만에 반등…日 수출규제 애먼 美에도 불똥

5th topic
[코스피 거래량 Top 10 :장중시황] 대창 6.32% 상승, 우성사료, 오리엔트바이오, 마니커, 백광소재 주가 순으로 5일 거래량 상위 기록.
키움증권, "SK하이닉스 3분기 영업익 1조원으로 올라설 것"
[마감] 트럼프, 미중 정상회담 시사…코스피, 상승 마감

6th topic
24일, 거래소 외국인 순매수상위에 금융업 업종 7종목
상반기 개인투자자 매수 상위 주식 10종목 중 9개 하락
24일, 거래소 기관 순매수상위에 금융업 업종 3종목

7th topic
삼성전자·SK하이닉스, 코스피 조정시 공매도 폭탄 우려
증시 하락에 한달새 대차거래 잔고 급증
코스피, 무역분쟁 불확실성 외인 매도 2060대ㆍ코스닥 7

In [11]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지

W shape: (298, 10)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 298 samples in 0.001s...
[t-SNE] Computed neighbors for 298 samples in 0.009s...
[t-SNE] Computed conditional probabilities for sample 298 / 298
[t-SNE] Mean sigma: 0.094289
[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.178360
[t-SNE] KL divergence after 1000 iterations: 0.178032


In [12]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [13]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})

p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [18]:
query = '일본'
query_index = word2id[query]  # 하락관련 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 SKT 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
ranks = query_vec.toarray().flatten().argsort()[::-1]

for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])

일본發 반도체 규제···일본 기업株도 반사이익 없이 ‘연일 하락’ http://www.sisajournal-e.com/news/articleView.html?idxno=202776
韓 투자로 6조 가까이 챙긴 日, 무역보복은 '자승자박' 악수 https://www.nocutnews.co.kr/news/5178143
日 수출 규제에도 반도체株 ‘덤덤’···반도체 소재주는 ‘들썩’ http://www.sisajournal-e.com/news/articleView.html?idxno=202531
[글로벌-Biz 24] 반도체 소재 생산 일본 기업들 對韓 수출규제로 주가 하락 http://www.g-enews.com/ko-kr/news/article/news_all/201907031334393602f806fa96c5_1/article.html
산 넘어 산인 투자환경…미중 무역분쟁에 이어 일본의 경제보복까지 https://news.imaeil.com/Economy/2019071218145619865
[日 경제보복]오르는 D램값…日 수출규제 美도 달갑지 않다 http://news.heraldcorp.com/view.php?ud=20190712000154
D램가격 10개월만에 반등…日 수출규제 애먼 美에도 불똥 http://news.heraldcorp.com/view.php?ud=20190712000294
일본 반도체 소재 수출 규제..."반도체·디스플레이주 영향 제한적" http://www.metroseoul.co.kr/news/newsview?newscd=2019070100192
日 경제보복 2주…롯데그룹 시총 7% 감소·SK는 증가 http://biz.chosun.com/site/data/html_dir/2019/07/14/2019071400334.html
日 경제보복 2주…롯데 시총 7% 감소·SK는 되레 늘어 http://news.chosun.com/site/data/html_dir/2019/07/13/2019071300889.html?utm_source=nave